In [ ]:
## Importing packages

# This R environment comes with all of CRAN and many other helpful packages preinstalled.
# You can see which packages are installed by checking out the kaggle/rstats docker image: 
# https://github.com/kaggle/docker-rstats

library(tidyverse) # metapackage with lots of helpful functions

## Running code

# In a notebook, you can run a single code cell by clicking in the cell and then hitting 
# the blue arrow to the left, or by clicking in the cell and pressing Shift+Enter. In a script, 
# you can run code by highlighting the code you want to run and then clicking the blue arrow
# at the bottom of this window.

## Reading in files

# You can access files from datasets you've added to this kernel in the "../input/" directory.
# You can see the files added to this kernel by running the code below. 

list.files(path = "../input")

## Saving data

# If you save any files or images, these will be put in the "output" directory. You 
# can see the output directory by committing and running your kernel (using the 
# Commit & Run button) and then checking out the compiled version of your kernel.

**#Objective: Predict Customer Life-time Value for an Auto Insurance Company
For an Auto Insurance company, predict the customer life time value (CLV). CLV is the total revenue the client will
derive from their entire relationship with a customer. Because we don't know how long each customer relationship will
be, we make a good estimate and state CLV as a periodic value — that is, we usually say “this customer's 12-month
(or 24-month, etc) CLV is $x”.**

In [ ]:
#Importing tidyverse and reading the data from source
library("tidyverse")
#data=read_csv("../input/ibm-watson-marketing-customer-value-data/WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv")
data<-read_csv("../input/data.csv")
class(data)

In [ ]:
#Importing relevant packages
library(caret)
library(ggplot2)
library(dplyr)
library(Hmisc)
library(broom)
library(glmnet)
library(pastecs)
library(psych)
library(tidyverse)
#library(funMoeling)

In [ ]:
#Here we are checking dimension and structure of the data
#1.Type of the data 2.RowXColumn 3.No. of variables and their type  
dim(data)
str(data)
#descriptive analysis: basic summary statistics
summary(data)
#glimpse(data)
#describe(data)

In [ ]:
#Analysis of Continuous variable
new_data<-data[sapply(data,is.numeric)]
#min-max-median-lower-upper hinge
fivenum(new_data$Income)
fivenum(new_data$Total.Claim.Amount)
fivenum(new_data$Monthly.Premium.Auto)
fivenum(new_data$Months.Since.Last.Claim)
fivenum(new_data$Number.of.Policies)
fivenum(new_data$Number.of.Open.Complaints)
fivenum(new_data$Months.Since.Policy.Inception)
#Univariate analysis<Quantitative measure>
#1.Measure of central tendency 2.Measure of dispersion 3.Skewness 4.Kurtosis
describe(new_data)
stat.desc(new_data)

In [ ]:
#Renaming columns
names(data)<-c("Customer","State","CLV","Response","Coverage","Education","Effective.To.Date","EmploymentStatus","Gender","Income","Location.Code","Marital.Status","Monthly.Premium.Auto","Months.Since.Last.Claim","Month.Since.Policy.Inception","Open.Complaints","Policies","Policy.Type","Policy","Renew.Offer.Type","Sales.Channel","Total.Claim.Amount","Vehicle.Class","Vehicle.Size")

In [ ]:
#Univariate analysis<Graphical measure>
#to understand the freuency of important categorical features and its distribution
data %>% ggplot(aes(x=Gender))+geom_bar(fill=c('#999999','#E69F00'))
data %>% ggplot(aes(x=as.factor(Marital.Status),fill=as.factor(Marital.Status)))+geom_bar()+scale_fill_hue(c = 40)+theme(legend.position="none")
data %>% ggplot(aes(x=as.factor(EmploymentStatus),fill=as.factor(EmploymentStatus)))+geom_bar()+scale_fill_brewer(palette = "Set1")+theme(legend.position="none")
data %>% ggplot(aes(x=as.factor(Location.Code),fill=as.factor(Location.Code)))+geom_bar()+scale_fill_brewer(palette = "Set2")+theme(legend.position="none")
data %>% ggplot(aes(x=as.factor(Policy),fill=as.factor(Policy)))+geom_bar()+scale_fill_grey(start = 0.25, end = 0.75)+theme(legend.position="none")
data %>% ggplot(aes(x=as.factor(Sales.Channel),fill=as.factor(Sales.Channel)))+geom_bar()

#important observations for continuous features<checking for outliers and other details>
boxplot(data$CLV)
boxplot(data$Monthly.Premium.Auto)
boxplot(data$Total.Claim.Amount)
boxplot(data$Income)

In [ ]:
#Bivariate analysis<Graphical measure>
#Categorical-Categorical
ggplot(data, 
       aes(x = Coverage, 
           fill = Education)) + 
  geom_bar(position = "stack")

ggplot(data, 
       aes(x = Coverage, 
           fill = Marital.Status)) + 
  geom_bar(position = "stack")

xtabs(~EmploymentStatus+Gender,data)
ggplot(data, 
       aes(x = EmploymentStatus, 
           fill = Location.Code)) + 
    geom_bar(position = position_dodge(preserve = "single"))

#Categorical-Continuous
ggplot(data)+geom_boxplot(aes(x=Income,y=factor(Gender)))

#Continuous-Continuous
data %>% ggplot(aes(x=Income, y=Monthly.Premium.Auto))+geom_point()+stat_smooth(method ="lm", se = FALSE)
data %>% ggplot(aes(x=Monthly.Premium.Auto, y=Total.Claim.Amount))+geom_point()+stat_smooth(method ="lm", se = FALSE)

In [ ]:
#chaeck for missing values
sum(is.na(data))
##Discarding insignificant features: Date and Customer ID(applying intuitive sense) 
data<-data[,-c(1,7)]
dim(data)

In [ ]:
#Bivariate analysis<Quantitative measure>
#subsetting with numerical variables
new_data<-subset(data,select=-c(CLV))
new_data<-new_data[sapply(new_data,is.numeric)]

#find correlation matrix
corr<-cor(new_data)
print(corr)
highlyCorrelated<-findCorrelation(corr, cutoff=0.5)
print(highlyCorrelated)
names<-colnames(highlyCorrelated)
names
library(corrplot)
corrplot(corr, order="FPC", method="circle", type="lower", tl.cex=0.7, tl.col=rgb(0,0,0))

In [ ]:
data1<-data
data2<-data
#Checking for outliers
boxplot(data2$CLV)#treatment require
boxplot(data2$Income)
boxplot(data2$Months.Since.Last.Claim)
boxplot(data2$Monthly.Premium.Auto)#treatment require
boxplot(data2$Policies)
boxplot(data2$Open.Complaints)
boxplot(data2$Month.Since.Policy.Inception)
boxplot(data2$Months.Since.Last.Claim)
boxplot(data2$Total.Claim.Amount)#treatment require

In [ ]:
#Removing outliers of Total.Claim.Amount
summary(data2$Total.Claim.Amount)
upper<-547.515+1.5*(547.515-272.258)
data2<-subset(data2,data2$Total.Claim.Amount<=upper)
#boxplot(data2$Total.Claim.Amount)
summary(data2$Total.Claim.Amount)
#2nd go
#dim(data2)
upper<-523.816+1.5*(523.816-256.438)
data2<-subset(data2,data2$Total.Claim.Amount<=upper)
boxplot(data2$Total.Claim.Amount)

In [ ]:
#Removing outliers of Monthly.Premium.Auto
boxplot(data2$Monthly.Premium.Auto)
summary(data2$Monthly.Premium.Auto)
u_range<-106+1.5*(106-68)
data2<-subset(data2,data2$Monthly.Premium.Auto<=u_range)
boxplot(data2$Monthly.Premium.Auto)
dim(data2)

In [ ]:
#Removing outliers of CLV
summary(data2$CLV)
u_range<-8609+1.5*(8609-3835)
data2<-subset(data2,data2$CLV<=u_range)
boxplot(data2$CLV)
dim(data2)
#2nd go
summary(data2$CLV)
u_range<-7956+1.5*(7956-3595)
data2<-subset(data2,data2$CLV<=u_range)
boxplot(data2$CLV)
dim(data2)
#3rd go
summary(data2$CLV)
u_range<-7842+1.5*(7842-3568)
data2<-subset(data2,data2$CLV<=u_range)
boxplot(data2$CLV)
dim(data2)

In [ ]:
#Creating dummy variables for categorical features and add CLV to new dataset
dmy<-dummyVars(CLV~.,data2,fullRank=TRUE)
X<-data.frame(predict(dmy,data2))
#Alternative way of train-test split
smp_size <- floor(0.80 * nrow(X))
ind <- sample(seq_len(nrow(X)), size = smp_size)
y<-as.data.frame(data2$CLV)
X_train <- X[ind, ]
X_test <- X[-ind, ]
y_train <- y[ind]
y_test<-y[-ind]

X$CLV<-data2$CLV
#scale data
preprocessParams<-preProcess(X, method = c("center", "scale"))
X <- predict(preprocessParams, X)
X$CLV<-data2$CLV
#check for dimension and structure
dim(X)
glimpse(X)

In [ ]:
#Importing relevant packages
library(rpart)
library(rpart.plot)
library(randomForest)
library(gbm)
library(caret)
#Create data partition using 80-20 rule
set.seed(999)
index<-createDataPartition(X$CLV, p=0.80, list=FALSE)
train<-X[index,]
test<-X[-index,]

In [ ]:
#Creating a custom function to obtain RMSE
calc_rmse<-function(actual,predicted){
    sqrt(mean(actual-predicted)^2)
}

In [ ]:
#Using a single tree
set.seed(999)
control<-trainControl(method="cv", number=10)
clv_tree = train(CLV~., train, method="rpart",trControl=control)
clv_tree_tst_pred = predict(clv_tree, newdata = test)
plot(clv_tree_tst_pred, test$CLV, 
     xlab = "Predicted", ylab = "Actual", 
     main = "Predicted vs Actual: Single Tree, Test Data",
     col = "dodgerblue", pch = 20)
grid()
abline(0, 1, col = "darkorange", lwd = 2)

In [ ]:
(tree_tst_rmse = calc_rmse(clv_tree_tst_pred, test$CLV))#3.34873575478434

In [ ]:
#Using caret package
#Done with cross-validation
#preprocessed parameter is being used 
set.seed(999)
control<-trainControl(method="cv", number=10)
clv_lm<-train(CLV~.,train, method="lm", metric="Rsquared", trControl=control, preprocess=c("center","scale","BoxCox"))#nzv,range,YeoJohnson
clv_lm_tst_pred<- predict(clv_lm, test)
plot(clv_lm_tst_pred, test$CLV,
    xlab= "Predicted", ylab= "Actual",
    main="Predicted vs Actual: Linear", col="blue", pch=18)
grid()
abline(0, 1, col = "red", lwd = 2)

In [ ]:
(lm_tst_rmse = calc_rmse(clv_lm_tst_pred, test$CLV))#14.4812044544613

In [ ]:
#Using lm()
set.seed(999)
clv_lm2<-lm(CLV~.,train)#nzv,range,YeoJohnson
clv_lm_tst_pred2<- predict(clv_lm2, test)
plot(clv_lm_tst_pred2, test$CLV,
    xlab= "Predicted", ylab= "Actual",
    main="Predicted vs Actual: Linear", col="blue", pch=18)
grid()
abline(0, 1, col = "red", lwd = 2)

In [ ]:
(lm_tst_rmse2 = calc_rmse(clv_lm_tst_pred2, test$CLV))#14.4812044544613

In [ ]:
#Bagging: a special case of Random Forest where mtry=no. of predictors
set.seed(999)
clv_bag<-randomForest(CLV~.,train,mtry=48,importance=TRUE,ntrees=500)
clv_bag
clv_bag_tst_pred<- predict(clv_bag, test)
plot(clv_bag_tst_pred, test$CLV,
    xlab= "Predicted", ylab= "Actual",
    main="Predicted vs Actual: Linear", col="blue", pch=18)
grid()
abline(0, 1, col = "red", lwd = 2)

In [ ]:
(bag_tst_rmse = calc_rmse(clv_bag_tst_pred, test$CLV))#19.3056638031333

In [ ]:
#Implementing Random Forest where I set mtry=no.of predictors/3
set.seed(999)
clv_rf<-randomForest(CLV~.,train,mtry=16,importance=TRUE,ntrees=500)
clv_rf
clv_rf_tst_pred<- predict(clv_rf, test)
plot(clv_rf_tst_pred, test$CLV,
    xlab= "Predicted", ylab= "Actual",
    main="Predicted vs Actual: Linear", col="blue", pch=18)
grid()
abline(0, 1, col = "red", lwd = 2)

In [ ]:
#Variable importance
importance(clv_rf,1)
varImpPlot(clv_rf,1)
#checking for RMSE
(rf_tst_rmse = calc_rmse(clv_rf_tst_pred, test$CLV))#12.3895307302228

In [ ]:
#prediction on train set
#clv_rf_trn_pred<-predict(clv_rf,train)
#(rf_trn_rmse<-calc_rmse(clv_rf_trn_pred,train$CLV))#4.50268351173869
(rf_oob_rmse<-calc_rmse(clv_rf$predicted,train$CLV))#10.538842165075

In [ ]:
#Random Forest with Hyper-parameter tuning
set.seed(999)
control<-trainControl(method="cv", number=10)
grid <- data.frame(.mtry = seq(2, 20, by =2))
clv_rf2 <- train(CLV~., train, method="rf", metric="Rsquared", trControl=control, preProcess=c("center","scale","BoxCox"), tuneGrid=grid)
summary(clv_rf2)
clv_rf2$finalModel
clv_rf2$results

In [ ]:
#Lasso regression
clv_lasso<-train(CLV~.,
                 data= train,
                 method = 'glmnet', 
                 tuneGrid = expand.grid(alpha = 1, lambda = 1))

clv_lasso_tst_pred<- predict(clv_lasso, test)
plot(clv_lasso_tst_pred, test$CLV,
    xlab= "Predicted", ylab= "Actual",
    main="Predicted vs Actual: Linear", col="blue", pch=18)
grid()
abline(0, 1, col = "red", lwd = 2)


In [ ]:
#after choosing regularization parameter
parameters <- c(seq(0.1, 2, by =0.1) ,  seq(2, 5, 0.5) , seq(5, 25, 1))

clv_lasso<-train(CLV~.,
             train,
                 method = 'glmnet', 
                 tuneGrid = expand.grid(alpha = 1, lambda = parameters) ,
                 metric =  "Rsquared"
               )
print(clv_lasso)
#test set RMSE
(lasso_tst_rmse = calc_rmse(clv_lasso_tst_pred, test$CLV))

In [ ]:
#Gradient Boosting Machine
clv_boost<-gbm(CLV~.,data=train,distribution="guassian",n.trees=5000,interaction.depth=4,shrinkage=0.01)
tibble::as_tibble(summary(clv_boost))

In [ ]:
set.seed(123)
control<-trainControl(method="repeatedcv", number=10, repeats=3)
clv_xgboost<-train(CLV~.,train,method="xgbTree",trControl=control,preProess=c("center","scale","BoxCox"))
#print(clv_xgboost)

In [ ]:
#Hyper-parameter tuning
set.seed(999)
grid<-expand.grid(nrounds=500, max_depth=seq(6,10),eta=c(0.01,0.3,1),gamma=c(0,0.2,1),
                  colsample_bytree=c(0.5,0.8,1),min_child_weight=seq(1,10))
ontrol<-trainControl(method="repeatedcv", number=10, repeats=3)
clv_xgboost<-train(CLV~.,train,method="xgbTree",trControl=control,preProess=c("center","scale","BoxCox"),tuneGrid=grid)
test_data<-xgb.DMatrix(test[,-c("CLV")])
pred<-predict(clv_xgboost,test_data)
rmse(log(test$target,log(pred)))

In [ ]:
#Variable importance
imp<-varImp(model2, scale=FALSE)
plot(imp)

In [ ]:
library(mlbench)
y_pred<-predict(model2,test)
postResample(pred = y_pred, obs = test$CLV)
attributes(model2)
model2$finalModel

In [ ]:
#Tuning hyperparameters
grid <- data.frame(.mtry = seq(2, 20, by =2))
rf2 <- model.rf<-train(CLV~CoveragePremium+EducationCollege+EmploymentStatusUnemployed+Income+Marital.StatusSingle+
              Monthly.Premium.Auto+Open.Complaints+Policies+
              Renew.Offer.TypeOffer2+Renew.Offer.TypeOffer3+Renew.Offer.TypeOffer4+Vehicle.ClassSports.Car+
              Vehicle.ClassSUV, train, method="rf", metric="Rsquared", trControl=control, preProcess=c("center","scale","BoxCox"), tuneGrid=tg)
summary(rf2)
rf2$results
#comparison check
model_list <- list(lm = model3, rf = rf2)
res <- resamples(model_list)
summary(res)
compare_models(model3, rf2)

In [ ]:
#Model Diagnostic and Assumption Check
#Linearity of the relationship between explanatory varibales and its target variable: residual vs fitted
#Error terms are normally distributed: Q-Q plot
plot(final_model)
#No correlation between variables: Autocorrelation Durbin-Watson
durbin.watson(final_model)
#Multicollinearity
corr<-cor(new_data)
print(corr)
highlyCorrelated<-findCorrelation(corr, cutoff=0.5)
print(highlyCorrelated)
names<-colnames(highlyCorrelated)
names
library(corrplot)
corrplot(corr, order="FPC", method="circle", type="lower", tl.cex=0.7, tl.col=rgb(0,0,0))
#alternative way of checking variable importance
#Previouly we have seen variable importance plot, checking p-value and statistical method- correlation plot
library(car)
vif(model1)

In [ ]:
#summary of the  model
#looking at adjusted R-squared and AIC-BIC
summary(model3)
summary(model3)$coeff
summary(model3)$r.squared
summary(model3)$adj.r.suared#'ll be more focus on that
#AIC(model3)
#BIC(model3)
#Model Diagnostic and Scoring
residuals<-resid(model3)
predValues<-predict(model3,test)
plot(test$CLV,residuals)
abline(0,0)
plot(test$CLV,predValues)

pred<-predict(model3,test)
actual_pred<-data.frame(cbind(test$CLV,pred))
cor(actual_pred)
head(actual_pred)

In [ ]:
#saveRDS(model,"https://drive.google.com/drive/my-drive/final_model.rds")